In [1]:
#!/usr/bin/env python

import argparse
import os
import os.path as osp
import torch
import models
from utils import get_log_dir, get_parameters
import dataset
from trainer import Trainer


configurations = {
    # same configuration as original work
    # https://github.com/shelhamer/fcn.berkeleyvision.org
    1: dict(
        max_iteration=100000,
        lr=1.0e-10,
        momentum=0.99,
        weight_decay=0.0005,
        interval_validate=4000,
    )
}


cfg = configurations[1]
out = get_log_dir('fcn8s-atonce', 1, cfg)
# resume= 'checkpoint'
resume = False
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# cuda = torch.cuda.is_available()
cuda = False
torch.manual_seed(1337)
if cuda:
    torch.cuda.manual_seed(1337)

In [2]:
# 1. dataset benchmark

root = '../datasets/'
kwargs = {'num_workers': 4, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    dataset.SBDClassSeg(root, split='train', transform=True),
    batch_size=1, shuffle=True, **kwargs)

val_loader = torch.utils.data.DataLoader(
    dataset.SBDClassSeg(root, split='val', transform=True),
    batch_size=1, shuffle=False, **kwargs)

# test_loader = torch.utils.data.DataLoader(
#     dataset.VOC2012ClassSeg(
#         root, split='seg11valid', transform=True),
#     batch_size=1, shuffle=False, **kwargs)



In [3]:
# 2. model

mod = models.FCN8sAtOnce(n_class=21)
start_epoch = 0
start_iteration = 0
if resume:
    checkpoint = torch.load(resume)
    mod.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint['epoch']
    start_iteration = checkpoint['iteration']
else:
    vgg16 = models.VGG16(pretrained=True)
    mod.copy_params_from_vgg16(vgg16)
if cuda:
    mod = mod.cuda()

# 3. optimizer

optim = torch.optim.SGD(
    [
        {'params': get_parameters(mod, bias=False)},
        {'params': get_parameters(mod, bias=True),
         'lr': cfg['lr'] * 2, 'weight_decay': 0},
    ],
    lr=cfg['lr'],
    momentum=cfg['momentum'],
    weight_decay=cfg['weight_decay'])
if resume:
    optim.load_state_dict(checkpoint['optim_state_dict'])



In [ ]:
# 3. optimizer
# optimizer_seg = torch.optim.Adam(seg.parameters(), lr=1e-3)
# optimizer_feat = torch.optim.Adam(res50.parameters(), lr=1e-4)


# optim = torch.optim.Adam(
#     [
#         {'params': get_parameters(mod, bias=False)},
#         {'params': get_parameters(mod, bias=True),
#          'lr': cfg['lr'] * 2, 'weight_decay': 0},
#     ],
#     lr=cfg['lr'],
# #     momentum=cfg['momentum'],
#     weight_decay=cfg['weight_decay'])



In [ ]:
trainer = Trainer(
    cuda=cuda,
    model=mod,
    optimizer=optim,
    train_loader=train_loader,
    val_loader=val_loader,
    out=out,
    max_iter=cfg['max_iteration'],
    interval_validate=cfg.get('interval_validate', len(train_loader)),
)
trainer.epoch = start_epoch
trainer.iteration = start_iteration
trainer.train()


Train epoch=0:   0%|                                   | 0/8498 [00:00<?, ?it/s]/home/lijilan/Documents/MR/segmentation/trainer.py:193: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if np.isnan(float(loss.data[0])):
/home/lijilan/Documents/MR/segmentation/utils.py:92: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/home/lijilan/Documents/MR/segmentation/utils.py:94: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
/home/lijilan/Documents/MR/segmentation/trainer.py:213: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  metrics.tolist() + [''] * 5 + [elapsed_time]

Train epoch=0:   0%|                        | 1/8498 [00:11<26:30:19, 11.23s/it]

[(0.99536, 0.3333333333333333, 0.3317866666666667, 0.9907415296000001)]



Train epoch=0:   0%|                        | 2/8498 [00:22<26:50:11, 11.37s/it]

[(0.933296, 0.25, 0.233324, 0.871041423616)]



Train epoch=0:   0%|                        | 3/8498 [00:33<26:06:10, 11.06s/it]

...

[(0.9274658753709198, 0.3560018742613184, 0.20189933801138865, 0.8729213134744378)]



Train epoch=0:  21%|███▉               | 1782/8498 [5:08:25<19:22:24, 10.38s/it]

[(0.5471938775510204, 0.3654109381317243, 0.2128137399943649, 0.3177257084970245)]



Train epoch=0:  21%|███▉               | 1783/8498 [5:08:35<19:22:13, 10.38s/it]

[(0.886592, 0.3626411036154396, 0.24360922227421467, 0.7930749049880782)]
